# Plan and Execute Pattern using Semantic Kernel

This notebook demonstrates the implementation of the Plan and Execute pattern using Semantic Kernel. This pattern improves agent performance by:

1. Breaking down complex tasks into manageable sub-tasks (Planning)
2. Executing each sub-task in sequence
3. Adapting to feedback during execution

## Architecture Overview

The Plan and Execute pattern involves:
- **Planner**: Responsible for generating a structured plan of sub-tasks
- **Executor**: Handles the execution of each sub-task
- **Memory**: Maintains context between steps
- **Tools**: Custom functions that can be called during execution

![Plan and Execute Pattern](../../../1_agentic-design-ptn/images/planning.png)

## Understanding the Plan and Execute Pattern

The Plan and Execute pattern we've demonstrated offers several advantages:

1. **Task Decomposition**: Complex tasks are broken down into simpler, manageable steps
2. **Tool Selection**: The planner automatically selects the appropriate tools for each step
3. **Adaptability**: If a step fails, the planner can adapt by trying alternative approaches
4. **Explainability**: The plan provides transparency into how the agent approaches problems

This pattern is particularly useful for tasks that require multiple steps or the use of various tools to complete.

## Practical Applications

The Plan and Execute pattern can be applied to various real-world scenarios:

- **Customer Support**: Helping customers troubleshoot complex issues step by step
- **Research Assistance**: Breaking down research questions into specific search queries and synthesis steps
- **Task Automation**: Creating workflows that combine multiple API calls and data transformations
- **Product Recommendations**: Gathering user preferences and matching them to suitable products

## Next Steps

To extend this pattern, consider:

1. Adding error handling to retry failed steps
2. Implementing dynamic replanning based on execution results
3. Incorporating user feedback between steps
4. Adding more specialized tools for specific domains

In [1]:
# Setup and Dependencies
import os
import json
import requests
import asyncio
import httpx
from urllib.parse import urljoin
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import List, Dict, Any
import nest_asyncio
from IPython.display import display, Markdown
import logging
import sys


parent_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from utils.search_utils import url_search

# Apply nest_asyncio to allow nested event loops (required for Jupyter)
nest_asyncio.apply()

# Configure logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

if logger.handlers:
    logger.handlers.clear()


handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(levelname)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.propagate = False

# Helper function to display markdown content
def display_md(text: str):
    display(Markdown(text))

In [2]:
# Load environment variables and configure Azure OpenAI
from dotenv import load_dotenv
load_dotenv()

# Get API keys from environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

# Check that environment variables are set
if not AZURE_OPENAI_ENDPOINT or not AZURE_OPENAI_API_KEY or not DEPLOYMENT_NAME:
    raise ValueError("Please set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, and AZURE_OPENAI_DEPLOYMENT_NAME in .env file")
    
print(f"Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Deployment Name: {DEPLOYMENT_NAME}")

# Check if Google Search API keys are available
if not GOOGLE_API_KEY or not GOOGLE_CSE_ID:
    print("Warning: GOOGLE_API_KEY or GOOGLE_CSE_ID not set. Web search functionality will be limited to demo mode.")

Azure OpenAI Endpoint: https://aoai-services1.openai.azure.com/
Deployment Name: gpt-4o-mini


In [3]:


async def extract_text_from_url(url):
    """
    Extract text content from a webpage URL.
    
    Args:
        url: The URL to extract text from
        
    Returns:
        Extracted text content
    """
    # For demo purposes, we'll generate some simulated content
    # In a real implementation, you would use httpx to fetch and parse the content
    
    # Simple simulation of webpage content based on URL
    if "example.com" in url:
        product_type = ""
        for product in ["smartphone", "laptop", "headphones", "smartwatch", "tablet"]:
            if product in url:
                product_type = product
                break
        
        if product_type:
            return f"This is a webpage about {product_type}s. It contains detailed information about features, specifications, and user reviews."
        else:
            return "This is a generic technology webpage with product information and reviews."
    
    return "Could not extract content from the provided URL."

async def add_context_async(urls):
    """
    Fetch and extract content from multiple URLs asynchronously.
    
    Args:
        urls: List of URLs to fetch content from
        
    Returns:
        List of extracted text content
    """
    tasks = [extract_text_from_url(url) for url in urls]
    results = await asyncio.gather(*tasks)
    return results

In [4]:
# Initialize Semantic Kernel with Azure OpenAI
kernel = sk.Kernel()

# Add Azure OpenAI service
kernel.add_service(
    AzureChatCompletion(
        service_id="azure_chat_completion",
        deployment_name=DEPLOYMENT_NAME,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY
    )
)

print("Semantic Kernel initialized with Azure OpenAI service")

Semantic Kernel initialized with Azure OpenAI service


## Creating Native Functions (Tools)

First, we'll create several native functions that our planner can use during execution. These functions represent the tools that our agent can use to complete tasks.

In [5]:
# Create a plugin with native functions that can be used by the planner
@kernel_function(
    description="Searches for information about a product.",
    name="search_product"
)
def search_product(query: str) -> str:
    """
    Searches for information about a specified product using web search.
    
    Args:
        query: The product to search for.
        
    Returns:
        Information about the product.
    """
    logger.info(f"Searching for product: {query}")
    
    # First try web search
    results = url_search(query=query, max_result=3, web_search_mode="bing")
    
    if results:
        # Process search results
        product_info = f"Found information about {query} from web search:\n\n"
        
        for i, result in enumerate(results):
            title = result.get("title", "Untitled")
            link = result.get("link", "")
            snippet = result.get("snippet", result.get("text_content", ""))
            
            product_info += f"{i+1}. {title}\n"
            product_info += f"   URL: {link}\n"
            if snippet:
                product_info += f"   Summary: {snippet}\n"
            product_info += "\n"
            
        return product_info
    
    return f"Could not find specific information about '{query}'."

@kernel_function(
    description="Compares features between two products.",
    name="compare_products"
)
def compare_products(product1: str, product2: str) -> str:
    """
    Compares features between two products using web search.
    
    Args:
        product1: First product to compare.
        product2: Second product to compare.
        
    Returns:
        Comparison between the products.
    """
    logger.info(f"Comparing products: {product1} vs {product2}")
    
    # Search for each product
    product1_results = url_search(query=product1, max_result=3, web_search_mode="bing")
    product2_results = url_search(query=product2, max_result=3, web_search_mode="bing")

    if product1_results and product2_results:
        comparison = f"Comparison between {product1} and {product2} based on web search:\n\n"
        
        # First product info
        comparison += f"## {product1.capitalize()} Information:\n"
        for i, result in enumerate(product1_results[:2]):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            comparison += f"{i+1}. {title}\n   {snippet}\n\n"
        
        # Second product info
        comparison += f"## {product2.capitalize()} Information:\n"
        for i, result in enumerate(product2_results[:2]):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            comparison += f"{i+1}. {title}\n   {snippet}\n\n"
        
        # Add comparison prompt
        comparison += f"Based on the information above, here are the key differences between {product1} and {product2}:"
        
        return comparison
    
    return f"No comparison data available between '{product1}' and '{product2}'. Try with more specific product types."

@kernel_function(
    description="Provides recommendations based on user preferences.",
    name="recommend_product"
)
def recommend_product(preferences: str) -> str:
    """
    Recommends products based on user preferences using web search.
    
    Args:
        preferences: User preferences for product recommendations.
        
    Returns:
        Product recommendations.
    """
    logger.info(f"Generating recommendations based on preferences: {preferences}")
    
    # Create a search query based on user preferences
    search_query = f"best products for {preferences}"
    
    # Perform web search
    results = url_search(query=search_query, max_result=3, web_search_mode="bing")
    
    if results:
        recommendations = f"Based on your preferences for '{preferences}', here are some recommended products:\n\n"
        
        for i, result in enumerate(results):
            title = result.get("title", "Untitled")
            snippet = result.get("snippet", result.get("text_content", "No description available"))
            
            recommendations += f"{i+1}. {title}\n"
            recommendations += f"   {snippet}\n\n"
            
        return recommendations
    
    return "Based on your preferences (from local database), I might need more specific information to make a tailored recommendation."

# Register functions with the kernel
# Register each function as a plugin
kernel.add_function(plugin_name="product_plugin", function=search_product)
kernel.add_function(plugin_name="product_plugin", function=compare_products)
kernel.add_function(plugin_name="product_plugin", function=recommend_product)

print("Native functions registered as tools")

Native functions registered as tools


## Creating Semantic Functions

Next, we'll create semantic functions that use natural language to perform tasks. These will be combined with our native functions in the planner.

In [6]:
# Create a semantic function for summarizing product information
summarize_prompt = """
You are a helpful product specialist.
Summarize the following product information in a concise and helpful manner.
Focus on the key features and benefits that would be most relevant to customers.

Product Information:
{{$input}}

Summary:
"""

summarize_function = kernel.add_function(
    function_name="summarize_product_info",
    plugin_name="product_plugin",
    prompt=summarize_prompt,
    description="Summarizes product information in a concise and helpful manner."
)

# Create a semantic function for generating product comparisons
compare_prompt = """
You are a helpful product specialist.
Create a detailed comparison between the products based on the information provided.
Include pros and cons for each product and mention which types of users would prefer each option.

Product Information:
{{$input}}

Comparison:
"""

compare_function = kernel.add_function(
    function_name="generate_detailed_comparison",
    plugin_name="product_plugin",
    prompt=compare_prompt,
    description="Generates a detailed comparison between products."
)

print("Semantic functions created")

Semantic functions created


## Executing a Plan

Let's put our plan and execute pattern to work with a sample scenario. We'll ask the agent to help a customer find the right product based on their needs.

In [7]:
# Configure the sequential planner
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.functions import KernelArguments


# request_settings = AzureChatPromptExecutionSettings(
#         # Force the AI model to choose the get_weather_for_city function for invocation.
#         function_choice_behavior=FunctionChoiceBehavior.Required(filters={"included_functions": ["search_product", "recommend_product"]}, max_auto_invoke_attempts=3),
# )

request_settings = AzureChatPromptExecutionSettings(
    function_choice_behavior=FunctionChoiceBehavior.Auto()
)

response = await kernel.invoke_prompt(
    "I need a recommendation for a product with good battery life for a college student.", 
    arguments=KernelArguments(settings=request_settings),
)
print(response)



INFO: Generating recommendations based on preferences: good battery life, college student
Here are some recommended resources for finding laptops with good battery life suitable for college students:

1. **The Best Laptops for College Students in 2025 - PCMag**
2. **Best Laptops for College Students 2025: Top Picks and Expert Advice**

These articles should provide a comprehensive overview of options available in the marketplace, highlighting models known for their battery performance. Would you like to look into specific models or features from these recommendations?


## Try different scenarios

Let's put our plan and execute pattern to work with a sample scenario. We'll ask the agent to help a customer find the right product based on their needs.

In [8]:
request_settings = AzureChatPromptExecutionSettings(
    function_choice_behavior=FunctionChoiceBehavior.Auto()
)

response = await kernel.invoke_prompt(
    "Compare the features of a samsung laptop and a samsung tablet for someone who needs to do both schoolwork and watch movies.", 
    arguments=KernelArguments(settings=request_settings),
)
print(response)

INFO: Searching for product: Samsung Laptop
INFO: Searching for product: Samsung Tablet
INFO: Searching for product: Samsung Galaxy Tablet
INFO: Generating recommendations based on preferences: Samsung laptop and tablet for schoolwork and movie watching
INFO: Comparing products: Samsung Galaxy Book5 Pro vs Samsung Galaxy Tab S8
INFO: Comparing products: Samsung Galaxy Book5 360 vs Samsung Galaxy Tab S8 Ultra
It seems I'm having trouble accessing specific information and comparisons for Samsung laptops and tablets. However, I can provide some general guidance on what you should look for in a Samsung laptop and tablet for schoolwork and movie watching:

### Samsung Laptop Features
1. **Performance**: Look for laptops with at least an Intel i5 or AMD Ryzen 5 processor, and a minimum of 8GB of RAM to handle multitasking.
2. **Display**: A Full HD (1920 x 1080) display is ideal for watching movies, providing clear and vibrant visuals.
3. **Storage**: SSD storage (256GB or higher) ensures qu